## RAG-basierter ChatBot zu den Parteiprogrammen zur Bundestagswahl 2025

In dieser Übung untersuchen wir die Parteiprogramme zur Bundestagswahl 2025 mithilfe von `langchain` und entwickeln einen ChatBot, dem wir Fragen zu den Programmen stellen können.

Im Moodle-Kurs finden Sie eine Datei `.env` mit einem API-Key für OpenAI. Bitte kopieren Sie die Datei in das Arbeitsverzeichnis bzw. legen Sie selbst eine Datei `.env` mit einem entsprechenden API-Key an:

```env
OPENAI_API_KEY=<YOUR OPENAI API KEY>
```

### Installation der benötigten Pakete

Neben diversen `langchain` Paketen benötigen wir `unstructured` zum Lesen der PDF-Dateien und ChromaDB als Vektordatenbank.

In [ ]:
!pip install  --upgrade langchain langchain-community langchain_chroma langchain_openai langchain_unstructured  unstructured[pdf] chromadb

In [ ]:
import chromadb
import os
from collections import defaultdict
from tqdm import tqdm
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_unstructured.document_loaders import UnstructuredLoader
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import MergerRetriever
from langchain.retrievers.document_compressors.flashrank_rerank import FlashrankRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from unstructured.partition.pdf import partition_pdf
from unstructured.chunking.title import chunk_by_title
from unstructured.chunking.basic import chunk_elements
from unstructured.documents.elements import Image

load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DATABASE_PATH = "./chroma/"
EMBEDDING_MODEL = "text-embedding-ada-002"

def pretty_output(chunks, mode: str):
    if mode == "elements":
        for i, chunk in enumerate(chunks, 1):
            print(f"Chunk {i}:")
            print(chunk.text)
            print("-" * 120)
            
    elif mode == "documents":
        for i, chunk in enumerate(chunks, 1):
            print(f"Chunk {i}:")
            print(chunk.page_content)
            print("-" * 120)

### Lesen der Wahlprogramme

Folgende Parteiprogramme liegen vor:

In [1]:
docs = {
    "BSW": "BSW_Parteiprogramm.pdf",
    "Grüne": "Grüne_BTW2025.pdf",
    "CDU": "CDU_BTW2025.pdf",
    "AfD": "Programm_AfD_Online_.pdf",
    "Linke": "DIE_LINKE_Wahlprogramm_zur_Bundestagswahl_2021.pdf",
    "SPD": "SPD-Zukunftsprogramm.pdf",
    "FDP": "fdp-wahlprogramm_2025.pdf"
}

In [ ]:
from os import path

# Chunker 2
max_characters = 5000
new_after_n_chars = 1500
overlap = 1000
combine_text_under_n_chars_multiplier=int(new_after_n_chars*(2/3))

DOCS = []

for (party, fpath) in docs.items():
    chunks = UnstructuredLoader(
        file_path=path.join("data", fpath),
        languages=["deu"],
        chunking_strategy="by_title",
        max_characters=max_characters,
        overlap=overlap,
        overlap_all=True,
        combine_text_under_n_chars=combine_text_under_n_chars_multiplier,
        new_after_n_chars=new_after_n_chars,
    ).load()

    # Füge Partei als Metadatenfeld hinzu
    for chunk in chunks:
        chunk.metadata["party"] = party
    DOCS += chunks


In [ ]:
len(DOCS), DOCS[-1]

Patchen der Metadaten: Chroma akzeptiert nur Strings als Metadaten, `unstructured` liefert allerdings eine *Liste* erkannter Sprachen. Diese Liste wandeln wir in einen String um.

In [ ]:
for chunk in DOCS:
    for md in chunk.metadata:
        if isinstance(chunk.metadata[md], list):
            chunk.metadata[md] = str(chunk.metadata[md])

### Speichern der Dokumente in ChromaDB

Mithilfe des Embedding-Modells speichern wir die Vektoren zu den Dokumenten.

**Bitte die nächste Zelle nur einmal ausführen**

In [ ]:
client = chromadb.PersistentClient(
    path=os.path.join(DATABASE_PATH, f"{EMBEDDING_MODEL}"),
)

Chroma.from_documents(
    documents=DOCS,
    embedding=OpenAIEmbeddings(api_key=OPENAI_API_KEY, model=EMBEDDING_MODEL),
    client=client,
    collection_name=f"BTW2025",
)

### Erstellen des Prompt Templates

Die folgende Zelle erstellt das `PromptTemplate` zur Beantwortung der Benutzerfragen.
Passen Sie das Template gerne nach Ihren Vorstellungen an.

In [ ]:
LLM = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-4o-mini",
    temperature=0.0,
)

PROMPT = ChatPromptTemplate([
    ("system", """Du bist ein Experte für politische Fragen zur Bundestagswahl und beantwortest die Fragen der Benutzer auf Basis des bereitgestellten Kontext. 
Der Kontext besteht aus eine Aufstellung der Aussagen einzelner Parteien zu der Fragestellung des Benutzers.

- Wenn die Frage anhand des Kontext beantwortet werden kann, gib in Deiner Antwort jeweils an, zu welcher Partei eine Aussage gehört.
- Wenn es Aussagen mehrerer Parteien gibt, stelle die Aussagen der Parteien gegenüber und verdeutliche die Unterschieder der Parteien.
- Wenn die Frage im Kontext nicht eindeutig beantwortet werden kann oder keine ausreichenden Informationen vorliegen, gib an, dass du die Frage nicht beantworten kannst.
- Achte besonders darauf, dass du keine Informationen hinzufügst, die nicht im Kontext enthalten sind.
- Gib am Ende Zitate aus den Aussagen der Parteien an, die Deine Zusammenfassung nachvollziehbar machen.

Wenn in der Frage nach der Position einer bestimmten Partei gefragt wird, gehe in der Antwort auf diese Partei ein.
Wenn in der Frage keine Partei explizit erwähnt wird, erstelle eine Übersicht der Positionen der folgenden Parteien:
- CDU
- SPD
- Grüne
- AfD
- FDP
- BSW
- Linke

Am Ende deiner Antwort weise bitte darauf hin, dass du ein ChatBot bist und die Antwort unbedingt anhand der Quellen überprüft werden sollte.

<kontext>
{context}
</kontext>"""),
    ("human", "Frage: {input}")
])


### Retriever

Der folgende Code erzeugt einen `Retriever`, der für jede Partei `k=3` zur Frage passende Aussagen sucht.

In [ ]:
vectorstore = Chroma(
        collection_name=f"BTW2025",
        client=client,
        embedding_function=OpenAIEmbeddings(model=EMBEDDING_MODEL, api_key=OPENAI_API_KEY),
        create_collection_if_not_exists=False
    )

PARTY_RETRIEVERS = [ 
    vectorstore.as_retriever(
        search_type='similarity',
        search_kwargs={
            'k': 3,
            'filter': {'party': party}
        }
    ) for party in docs.keys() ]

lotr = MergerRetriever(retrievers=PARTY_RETRIEVERS)


### RetrievalChain

Die folgende *Retrieval Chain* erzeugt mithilfe des Retrievers und des ChatPromptTemplates eine Antwort auf eine Benutzerfrage.

In [ ]:
retrieval_chain = create_retrieval_chain(
    retriever=lotr,
    combine_docs_chain=create_stuff_documents_chain(
        llm=LLM,
        prompt=PROMPT,
        document_prompt=PromptTemplate.from_template("{party}: {page_content}")
    )
)

Testen Sie die Chain mit eigenen Fragen!

In [ ]:
retrieval_chain.invoke({"input": "Was ist die Haltung der Parteien zur Schuldenbremse und zu Investitionen?"})